In [ ]:
import pandas as pd
from openpyxl import load_workbook
import difflib

# Function to load comments from an Excel file
def extract_comments(file_path):
    wb = load_workbook(file_path)
    ws = wb.active
    comments = {}
    
    for row in ws.iter_rows():
        for cell in row:
            if cell.comment:
                comments[cell.coordinate] = cell.comment.text
    return comments

# Function to load the knowledge base and add comments
def load_knowledge_base_with_comments(file_path):
    # Load the Excel file into a DataFrame
    if file_path == '/content/ClientABC _ ATB Financial_Final Version.xlsx':
      df = pd.read_excel(file_path, header=5)
      # Extract comments from the Excel file
      comments = extract_comments(file_path)
      # Add comments to the DataFrame
      df['comment'] = df.apply(lambda row: comments.get(f'{row.name+6}', ''), axis=1)
      return df
    else:
      df = pd.read_excel(file_path)
      # Extract comments from the Excel file
      comments = extract_comments(file_path)
      # Add comments to the DataFrame
      df['comment'] = df.apply(lambda row: comments.get(f'{row.name+2}', ''), axis=1)
      return df

# Load the two versions of the questionnaire
file_path_v1 = '/content/ClientABC _ ATB Financial_Version1.xlsx'
file_path_final = '/content/ClientABC _ ATB Financial_Final Version.xlsx'

# Load DataFrames with comments
df_v1 = load_knowledge_base_with_comments(file_path_v1)
df_final = load_knowledge_base_with_comments(file_path_final)


In [ ]:
# Function to compare two DataFrames
def compare_questionnaires(df_v1, df_final):
    differences = []
    
    for i in range(len(df_v1)):
        s1 = str(df_v1.iloc[i]['Section Heading']) + ' ' + df_v1.iloc[i]['comment'] 
        c1 = str(df_v1.iloc[i]['Control Heading']) + ' ' + df_v1.iloc[i]['comment'] 
        q1 = str(df_v1.iloc[i]['Question Text']) + ' ' + df_v1.iloc[i]['comment'] 
        a1 = str(df_v1.iloc[i]['Answer']) + ' ' + df_v1.iloc[i]['comment'] 
        n1 = str(df_v1.iloc[i]['Notes/Comment']) + ' ' + df_v1.iloc[i]['comment'] 

        s2 = str(df_final.iloc[i]['Section Heading']) + ' ' + df_v1.iloc[i]['comment']
        c2 = str(df_final.iloc[i]['Control Heading']) + ' ' + df_v1.iloc[i]['comment']
        q2 = str(df_final.iloc[i]['Question Text']) + ' ' + df_v1.iloc[i]['comment'] 
        a2 = str(df_final.iloc[i]['Answer']) + ' ' + df_final.iloc[i]['comment'] 
        n2 = str(df_v1.iloc[i]['Notes/Comment']) + ' ' + df_v1.iloc[i]['comment'] 
        
        if q1 != q2 or a1 != a2 or n1 != n2:
            diff = difflib.unified_diff(q1.split() + a1.split(), q2.split() + a2.split(), fromfile='Version 1', tofile='Final Version')
            differences.append('\n'.join(diff))
    
    return differences

# Compare the two DataFrames
differences = compare_questionnaires(df_v1, df_final)

# Print the differences
for diff in differences:
    pprint(diff)

In [ ]:
import pandas as pd
import difflib
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:

# Analyze the differences using NLP
def analyze_differences(differences):
    # Tokenize the differences
    tokens = [nltk.word_tokenize(diff) for diff in differences]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [[word for word in token if word.lower() not in stop_words] for token in tokens]
    
    # Count the frequency of words
    word_freq = {}
    for token in filtered_tokens:
        for word in token:
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1
    
    # Generate a word cloud
    wordcloud = WordCloud().generate_from_frequencies(word_freq)
    plt.figure(figsize=(8,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

analyze_differences(differences)

In [ ]:

# Visualize the differences
def visualize_differences(differences):
    # Create a bar chart of the number of differences per question
    diff_counts = [len(diff.split('\n')) for diff in differences]
    plt.figure(figsize=(10,6))
    plt.bar(range(len(diff_counts)), diff_counts)
    plt.xticks(range(len(diff_counts)), range(1, len(diff_counts)+1))
    plt.xlabel('Question')
    plt.ylabel('Number of Differences')
    plt.title('Differences between Questionnaire Versions')
    plt.show()

visualize_differences(differences)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load datasets
file_path_v1 = '/content/ClientABC _ ATB Financial_Version1.xlsx'
file_path_final = '/content/ClientABC _ ATB Financial_Final Version.xlsx'

df_v1 = pd.read_excel(file_path_v1)
df_final = pd.read_excel(file_path_final, header=5)

# Visualize Version 1
plt.figure(figsize=(10, 6))
sns.heatmap(df_v1.isnull(), cbar=False, cmap='viridis')
plt.title('Version 1 - Missing Values Heatmap')
plt.show()

# Visualize Final Version
plt.figure(figsize=(10, 6))
sns.heatmap(df_final.isnull(), cbar=False, cmap='viridis')
plt.title('Final Version - Missing Values Heatmap')
plt.show()

# Ensure the indices are aligned
df_v1.reset_index(drop=True, inplace=True)
df_final.reset_index(drop=True, inplace=True)

# Combine datasets
combined_df = pd.concat([df_v1.add_suffix('_v1'), df_final.add_suffix('_final')], axis=1)
combined_df.head()

# Highlight the differences
def highlight_differences(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs(1, axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

# Apply the function
styled_combined_df = combined_df.style.apply(highlight_differences, axis=None)
styled_combined_df

# Count the differences per column
diff_count = (df_v1 != df_final).sum().reset_index()
diff_count.columns = ['Column', 'Differences']

# Plot the differences
plt.figure(figsize=(12, 8))
sns.barplot(x='Column', y='Differences', data=diff_count)
plt.xticks(rotation=90)
plt.title('Number of Differences per Column')
plt.show()


In [ ]:
import pandas as pd

# Load datasets
file_path_v1 = '/content/ClientABC _ ATB Financial_Version1.xlsx'
file_path_final = '/content/ClientABC _ ATB Financial_Final Version.xlsx'

df_v1 = pd.read_excel(file_path_v1)
df_final = pd.read_excel(file_path_final, header=5)

# Ensure the indices are aligned
df_v1.reset_index(drop=True, inplace=True)
df_final.reset_index(drop=True, inplace=True)

# Identify the differences
diff_mask = df_v1 != df_final

# Iterate through columns and print differences
for column in df_v1.columns:
    differences = diff_mask[column]
    if differences.any():
        print(f"Differences in column: {column}")
        for i, diff in enumerate(differences):
            if diff:
                print(f"Row {i+1}: Version 1 = {df_v1.at[i, column]}, Final Version = {df_final.at[i, column]}")
        print("\n")
